In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import timm
from torchvision import transforms as T
from tqdm import tqdm
import gc

class CFG:
    # General
    num_workers = 4
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Data paths
    data_dir = './'
    labels_csv_path = os.path.join(data_dir, 'labels.csv')
    sample_submission_path = os.path.join(data_dir, 'sample_submission.csv')
    test_img_dir = os.path.join(data_dir, 'test')
    # MODIFIED: Use blended thresholds
    thresholds_path = 'blended_thresholds.npy'

    # Model
    model_name = 'tf_efficientnet_b4_ns'
    img_size = 384
    model_paths = [
        'models/tf_efficientnet_b4_ns_fold0_best.pth',
        'models/tf_efficientnet_b4_ns_fold1_best.pth',
        'models/tf_efficientnet_b4_ns_fold2_best.pth'
    ]

    # Inference
    # FIX 2: Reduce batch size further to 8 to prevent OOM error.
    batch_size = 8

# Load label mappings
labels_df = pd.read_csv(CFG.labels_csv_path)
CFG.attr_ids = labels_df['attribute_id'].values
CFG.attr_id_to_idx = {attr_id: i for i, attr_id in enumerate(CFG.attr_ids)}
CFG.idx_to_attr_id = {i: attr_id for i, attr_id in enumerate(CFG.attr_ids)}
CFG.num_classes = len(labels_df)

# Clean up memory
torch.cuda.empty_cache()
gc.collect()

0

In [2]:
def get_test_transforms(flipped=False):
    # This uses aspect-ratio preserving resize, center crop, and ImageNet normalization.
    print("--- Applying CORRECTED validation transforms (Resize+CenterCrop, ImageNet Norm) ---")
    
    transforms_list = [
        T.Resize(CFG.img_size), # Preserves aspect ratio
        T.CenterCrop(CFG.img_size),
    ]
    
    if flipped:
        transforms_list.append(T.RandomHorizontalFlip(p=1.0))

    transforms_list.extend([
        T.ToTensor(),
        T.Normalize(
            mean=[0.485, 0.456, 0.406], # ImageNet stats
            std=[0.229, 0.224, 0.225],
        ),
    ])
    
    return T.Compose(transforms_list)

class iMetTestDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.filepaths = df['filepath'].values
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        filepath = self.filepaths[idx]
        image = Image.open(filepath).convert('RGB')
        
        if self.transforms:
            image = self.transforms(image)
            
        return image

class iMetModel(nn.Module):
    def __init__(self, model_name, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, num_classes=CFG.num_classes)

    def forward(self, x):
        return self.model(x)

In [3]:
## Prepare Test Data
sub_df = pd.read_csv(CFG.sample_submission_path)
sub_df['filepath'] = sub_df['id'].apply(lambda x: os.path.join(CFG.test_img_dir, x + '.png'))
display(sub_df.head())

# Create two datasets: one for original images, one for flipped
test_dataset_normal = iMetTestDataset(sub_df, transforms=get_test_transforms(flipped=False))
test_loader_normal = DataLoader(test_dataset_normal, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)

test_dataset_flipped = iMetTestDataset(sub_df, transforms=get_test_transforms(flipped=True))
test_loader_flipped = DataLoader(test_dataset_flipped, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)

,id,attribute_ids,filepath
0,347c119163f84420f10f7a8126c1b8a2,0 1 2,./test/347c119163f84420f10f7a8126c1b8a2.png
1,98c91458324cba5415c5f5d8ead68328,0 1 2,./test/98c91458324cba5415c5f5d8ead68328.png
2,3f75d332f579af62ff88d369c0736c76,0 1 2,./test/3f75d332f579af62ff88d369c0736c76.png
3,3fa35a29218b7449c8f03e2a368a708d,0 1 2,./test/3fa35a29218b7449c8f03e2a368a708d.png
4,c848b91558e4edd8034cb7d334b4e448,0 1 2,./test/c848b91558e4edd8034cb7d334b4e448.png


--- Applying CORRECTED validation transforms (Resize+CenterCrop, ImageNet Norm) ---
--- Applying CORRECTED validation transforms (Resize+CenterCrop, ImageNet Norm) ---


In [6]:
## Inference with TTA (Ensemble) - MEMORY-EFFICIENT VERSION

# Pre-allocate array for summed predictions for memory efficiency
n_samples = len(sub_df)
# We will sum predictions from 3 models * 2 augmentations (normal, flipped)
total_preds = np.zeros((n_samples, CFG.num_classes), dtype=np.float32)
tta_loaders = {
    "normal": test_loader_normal,
    "flipped": test_loader_flipped
}

for i, model_path in enumerate(CFG.model_paths):
    print(f"--- Inferencing with model {i+1}/{len(CFG.model_paths)}: {model_path} ---")
    
    # FIX for OOM: Create model on CPU, load weights, THEN move to GPU.
    model = iMetModel(CFG.model_name, pretrained=False) # 1. Create on CPU
    state_dict = torch.load(model_path, map_location='cpu', weights_only=True) # 2. Load weights to CPU
    model.load_state_dict(state_dict)
    model.to(CFG.device) # 3. Move fully loaded model to GPU
    model.eval()

    # TTA loop
    for tta_type, test_loader in tta_loaders.items():
        print(f"  -- TTA: {tta_type} --")
        pbar = tqdm(test_loader, desc=f"Predicting (Model {i+1}, {tta_type})")
        current_pos = 0
        with torch.no_grad():
            for images in pbar:
                images = images.to(CFG.device)
                logits = model(images)
                preds = logits.sigmoid().cpu().numpy()
                
                batch_size = images.size(0)
                total_preds[current_pos : current_pos + batch_size] += preds
                current_pos += batch_size
            
    # Clean up memory after each model
    del model, state_dict
    torch.cuda.empty_cache()
    gc.collect()

# Average the predictions (3 models * 2 TTA = 6 total predictions per image)
all_preds = total_preds / (len(CFG.model_paths) * len(tta_loaders))
print("\nEnsemble TTA predictions calculated.")

--- Inferencing with model 1/3: models/tf_efficientnet_b4_ns_fold0_best.pth ---


  -- TTA: normal --


Predicting (Model 1, normal):   0%|          | 0/2665 [00:00<?, ?it/s]

Predicting (Model 1, normal):   0%|          | 1/2665 [00:00<08:44,  5.08it/s]

Predicting (Model 1, normal):   0%|          | 4/2665 [00:00<02:59, 14.81it/s]

...

Predicting (Model 3, flipped): 100%|█████████▉| 2659/2665 [01:38<00:00, 27.08it/s]

Predicting (Model 3, flipped): 100%|█████████▉| 2662/2665 [01:38<00:00, 27.11it/s]

Predicting (Model 3, flipped): 100%|██████████| 2665/2665 [01:38<00:00, 27.65it/s]

Predicting (Model 3, flipped): 100%|██████████| 2665/2665 [01:38<00:00, 27.04it/s]


Ensemble TTA predictions calculated.


In [7]:
## Create Submission with Blended Thresholds

# Load the blended thresholds
thresholds = np.load(CFG.thresholds_path)
print(f"Loaded blended thresholds from: {CFG.thresholds_path}")
print(f"Thresholds shape: {thresholds.shape}")

predictions = []
for pred_row in tqdm(all_preds, desc="Formatting submission"):
    # Apply blended per-class thresholds
    pred_labels = (pred_row > thresholds).astype(int)
    
    # If no labels are predicted, take the one with the highest probability as a fallback
    if pred_labels.sum() == 0:
        pred_labels[pred_row.argmax()] = 1
        
    # Convert indices to attribute_ids
    attr_ids = [CFG.idx_to_attr_id[i] for i, label in enumerate(pred_labels) if label == 1]
    predictions.append(' '.join(map(str, attr_ids)))

sub_df['attribute_ids'] = predictions
sub_df[['id', 'attribute_ids']].to_csv('submission.csv', index=False)
print("Submission file created successfully!")
display(sub_df.head())

Loaded blended thresholds from: blended_thresholds.npy
Thresholds shape: (3474,)


Formatting submission:   0%|          | 0/21318 [00:00<?, ?it/s]

Formatting submission:   2%|▏         | 463/21318 [00:00<00:04, 4621.61it/s]

Formatting submission:   4%|▍         | 928/21318 [00:00<00:04, 4637.45it/s]

Formatting submission:   7%|▋         | 1392/21318 [00:00<00:04, 4625.16it/s]

...

Formatting submission:  96%|█████████▋| 20537/21318 [00:04<00:00, 4518.22it/s]

Formatting submission:  98%|█████████▊| 20989/21318 [00:04<00:00, 4500.30it/s]

Formatting submission: 100%|██████████| 21318/21318 [00:04<00:00, 4526.98it/s]

Submission file created successfully!


,id,attribute_ids,filepath
0,347c119163f84420f10f7a8126c1b8a2,424 430 457 571 584 587 641 737 1020 1071 1239...,./test/347c119163f84420f10f7a8126c1b8a2.png
1,98c91458324cba5415c5f5d8ead68328,1283 2635 2658 2663 2715 2730 2737 2859 2968 3...,./test/98c91458324cba5415c5f5d8ead68328.png
2,3f75d332f579af62ff88d369c0736c76,1234,./test/3f75d332f579af62ff88d369c0736c76.png
3,3fa35a29218b7449c8f03e2a368a708d,11 55 95 421 450 693 734 773 833 924 935 1254 ...,./test/3fa35a29218b7449c8f03e2a368a708d.png
4,c848b91558e4edd8034cb7d334b4e448,946 1537,./test/c848b91558e4edd8034cb7d334b4e448.png
